# Text to SQL

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "PASTE YOUR API KEY HERE"
GEMINI_API_KEY = "PASTE YOUR API KEY HERE"


In [2]:
! pip install langchain-google-genai
! pip install langchain-core


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import sqlite3
import os



from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


In [4]:
if os.path.exists('employee_management.db'):
    os.remove('employee_management.db')

# Create a connection
conn = sqlite3.connect('employee_management.db')
cursor = conn.cursor()

# Table for departments
print("Creating departments table...")
cursor.execute('''
CREATE TABLE departments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    location TEXT
)
''')

# Table for jobroles
print("Creating jobroles table...")
cursor.execute('''
CREATE TABLE jobroles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    job_title TEXT NOT NULL,
    min_salary REAL,
    max_salary REAL
)
''')

# Table for employees
print("Creating employees table...")
cursor.execute('''
CREATE TABLE employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    department_id INTEGER,
    job_title TEXT,
    salary REAL,
    hire_date TEXT,
    FOREIGN KEY(department_id) REFERENCES departments(id)
)
''')


# Insert data into Department Table
print("Inserting data into departments...")
departments_data = [
    ('Engineering', 'New York'),
    ('Human Resources', 'Los Angeles'),
    ('Sales', 'Chicago')
]
cursor.executemany('INSERT INTO departments (name, location) VALUES (?, ?)', departments_data)

# Insert data into JobRole Table
print("Inserting data into jobroles...")
job_roles_data = [
    ('Software Engineer', 70000, 120000),
    ('HR Manager', 60000, 90000),
    ('Sales Executive', 50000, 80000)
]
cursor.executemany('INSERT INTO jobroles (job_title, min_salary, max_salary) VALUES (?, ?, ?)', job_roles_data)

# Insert data into Employee Table
print("Inserting data into employees...")
employees_data = [
    ('Alice', 'Smith', 1, 'Software Engineer', 85000, '2022-05-01'),
    ('Bob', 'Brown', 2, 'HR Manager', 75000, '2021-03-15'),
    ('Charlie', 'Davis', 3, 'Sales Executive', 65000, '2023-01-10'),
    ('Diana', 'Miller', 1, 'Software Engineer', 95000, '2023-08-20') # Added another engineer for a better example
]
cursor.executemany('INSERT INTO employees (first_name, last_name, department_id, job_title, salary, hire_date) VALUES (?, ?, ?, ?, ?, ?)', employees_data)

conn.commit()
conn.close()

print("Database setup complete!")

Creating departments table...
Creating jobroles table...
Creating employees table...
Inserting data into departments...
Inserting data into jobroles...
Inserting data into employees...
Database setup complete!


In [5]:
try:
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        google_api_key=GEMINI_API_KEY,
        temperature=0.3, # Lower temperature for more predictable SQL generation
        max_tokens=500
    )
except Exception as e:
    print(f"Error initializing the language model: {e}")
    print("Please ensure you have set your GOOGLE_API_KEY correctly.")
    llm = None

def call_gemini_with_langchain(prompt):
    """Generic function to call the Gemini API using LangChain."""
    if not llm:
        return "Language model is not initialized."
    try:
        # We wrap the prompt in a HumanMessage object
        message = HumanMessage(content=prompt)
        response = llm.invoke([message])
        print(response.content)
        return response.content
    except Exception as e:
        print(f"Error calling Gemini API via LangChain: {e}")
        return None

def text_to_sql(question):
    """Translates a natural language question into an SQL query using Gemini."""
    table_info = """
    The database contains the following tables:

    1. employees table:
    - id (INTEGER PRIMARY KEY): Employee ID
    - first_name (TEXT): First Name
    - last_name (TEXT): Last Name
    - department_id (INTEGER): Foreign Key referencing departments.id
    - job_title (TEXT): Job Position
    - salary (REAL): Employee Salary
    - hire_date (TEXT): Date of Hiring (format: YYYY-MM-DD)
    \n
    2. departments table:
    - id (INTEGER PRIMARY KEY): Department ID
    - name (TEXT): Department Name
    - location (TEXT): Department Location
    \n
    3. jobroles table:
    - id (INTEGER PRIMARY KEY): Job Role ID
    - job_title (TEXT): Job Title
    - min_salary (REAL): Minimum Salary for the Job
    - max_salary (REAL): Maximum Salary for the Job
    """

    prompt = f"""
    You are an expert that translates natural language questions into SQL queries.
    Based on the database schema below, translate the following question into a single, valid SQLite query.
    Return only the SQL query itself, with no explanations, introductory text, or markdown formatting.

    ---
    SCHEMA:
    {table_info}
    ---
    QUESTION: {question}
    ---
    SQL QUERY:
    """

    sql_query = call_gemini_with_langchain(prompt)
    if sql_query:
        # Clean up the response to ensure it's just the query
        return sql_query.strip().replace('```sql', '').replace('```', '').strip()
    return None


In [6]:

def run_sql_query(query):
    """Executes an SQL query and returns the results."""
    try:
        conn = sqlite3.connect('employee_management.db')
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        columns = [description[0] for description in cursor.description] if cursor.description else []
        conn.close()
        return columns, results
    except sqlite3.Error as e:
        return None, f"SQL Error: {e}"

def process_results_with_llm(question, columns, results):
    """Summarizes the SQL query results into a human-readable format using Gemini."""
    if results is None:
        return "An error occurred while executing the query."

    if isinstance(results, str):
        return results

    if not results:
        return "The query returned no results."

    formatted_results = "\n".join([str(dict(zip(columns, row))) for row in results])

    prompt = f"""
    You are a helpful assistant that summarizes database query results in a user-friendly way.
    A user asked the following question: "{question}"

    The following results were retrieved from the database:
    {formatted_results}

    Please provide a concise and natural language summary of these results that directly answers the user's question.
    Do not mention SQL or the database. Just present the answer.
    """

    summary = call_gemini_with_langchain(prompt)
    return summary if summary else "Failed to generate a summary for the results."


In [7]:
def final_process_sql_query(question):
    """The main function to orchestrate the text-to-SQL and summarization process."""
    if not llm:
        return "Cannot process question because the language model is not available."

    print(f"\nProcessing question: '{question}'")

    print("1. Translating question to SQL...")
    sql_query = text_to_sql(question)
    print(f"   - Generated SQL query: {sql_query}")

    if not sql_query:
        return "Failed to generate SQL query."
    print(f"   - Generated SQL: {sql_query}")

    print("2. Executing SQL query...")
    columns, results = run_sql_query(sql_query)
    print(f"   - Query executed. Found {len(results) if isinstance(results, list) else 'error' } records.")

    print("3. Summarizing results...")
    final_answer = process_results_with_llm(question, columns, results)
    print("   - Summary generated.")
    print(f"   - Final answer: {final_answer}")
    return final_answer


In [8]:
question = "Which location's employee earns the most, and what is their salary?"
final_result = final_process_sql_query(question)
print("\nFinal Result:",final_result)


Processing question: 'Which location's employee earns the most, and what is their salary?'
1. Translating question to SQL...


SELECT T1.location, T2.salary FROM departments AS T1 INNER JOIN employees AS T2 ON T1.id = T2.department_id ORDER BY T2.salary DESC LIMIT 1
   - Generated SQL query: SELECT T1.location, T2.salary FROM departments AS T1 INNER JOIN employees AS T2 ON T1.id = T2.department_id ORDER BY T2.salary DESC LIMIT 1
   - Generated SQL: SELECT T1.location, T2.salary FROM departments AS T1 INNER JOIN employees AS T2 ON T1.id = T2.department_id ORDER BY T2.salary DESC LIMIT 1
2. Executing SQL query...
   - Query executed. Found 1 records.
3. Summarizing results...
The employee earning the most is located in New York, with a salary of $95,000.
   - Summary generated.
   - Final answer: The employee earning the most is located in New York, with a salary of $95,000.

Final Result: The employee earning the most is located in New York, with a salary of $95,000.


In [9]:
print("\nFinal Result:",final_result)


Final Result: The employee earning the most is located in New York, with a salary of $95,000.


### WITH Agent Executer ( Not providing the schema , direct the database) - (OPTIONAL)

In [10]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

db_file = 'employee_management.db'
db_uri = f"sqlite:///{db_file}"
db = SQLDatabase.from_uri(db_uri, engine_args={"connect_args": {"check_same_thread": False}})


# Create the SQL Agent
if llm:
    agent_executor = create_sql_agent(
        llm=llm,
        db=db,
        agent_type="openai-tools",
        # verbose=True
        verbose=False )
else:
    agent_executor = None


def ask_agent(question):
    """Invokes the agent with a question and prints the result."""
    if not agent_executor:
        print("Agent is not initialized. Cannot process the question.")
        return

    print(f"\n--- Asking Agent: '{question}' ---")
    try:
        result = agent_executor.invoke({"input": question})
        print("\n--- Final Answer ---")
        print(result['output'])
    except Exception as e:
        print(f"An error occurred while running the agent: {e}")


# Run examples
ask_agent("Which location's employee earns the most, and what is their salary?")
# ask_agent("How many people work in the Engineering department?")
# ask_agent("List all software engineers and their salaries, ordered from highest to lowest salary.")




--- Asking Agent: 'Which location's employee earns the most, and what is their salary?' ---

--- Final Answer ---
The employee with the highest salary is located in New York, and their salary is $95000.0.

